In [1]:
import os 
kafka = "org.apache.spark:spark-sql-kafka-0-10_2.12:3.0.1"
spark = "org.apache.spark:spark-streaming-kafka-0-8_2.11:2.1.0"

os.environ["PYSPARK_SUBMIT_ARGS"] = ("--packages {},{} pyspark-shell".format(kafka, spark))

from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
from functools import reduce
from pyspark.sql import DataFrame
from pyspark.sql.functions import *
from pyspark.sql.types import *

KAFKA_BROKER = "kafka:9092"
KAFKA_TOPIC = "fixer"

sc = SparkContext('local')
spark = SparkSession(sc)

# Đọc dữ liệu từ Kafka bằng Spark Streaming

In [2]:
kafkaMessages = spark \
  .readStream \
  .format("kafka") \
  .option("kafka.bootstrap.servers", KAFKA_BROKER) \
  .option("subscribe", KAFKA_TOPIC) \
  .option("startingOffsets", "earliest") \
  .load()

# Lấy value của dữ liệu từ kafka và convert từ binary sang String

In [3]:
message = kafkaMessages.selectExpr("CAST(value AS STRING)")

In [4]:
rawQuery = message \
        .writeStream \
        .queryName("qraw1")\
        .format("memory")\
        .start()

# Ghi dữ liệu vào HDFS dùng Spark Streaming

In [ ]:
fileStream = message.writeStream \
                      .trigger(processingTime='60 seconds')\
                      .queryName("Persist the processed data") \
                      .outputMode("append") \
                      .format("json") \
                      .option("path", "hdfs://namenode/user/root/inputHDFS/") \
                      .option("checkpointLocation", "hdfs://namenode/user/root/checkpoint") \
                      .start() \
                      .awaitTermination()